In [1]:
from nilearn import connectome, plotting, datasets
import pickle
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from esfMRI import average_time_series, save_fc, align

In [2]:
atlas_labels = datasets.fetch_atlas_aal()['labels']

In [6]:
# with open("silced_windows.pkl", "rb") as f:
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [7]:
data["sub-292"]["ses-preop"]["run-01"]["time_series"].shape

(179, 116)

In [8]:
FCs = {} # 所有数据计算得到的功能连接
for sub, ses in data.items():
    FCs[sub] = {}
    for op, runs in ses.items():
        FCs[sub][op] = {}
        for run, items in runs.items():
            # if len(items["sliced_time_series"]) == 0:
            #     print(sub, op, run, items["time_series"].shape)
            #     continue
            # FCs[sub][op][run] = {
            #     "entire" : connectome.ConnectivityMeasure(kind="correlation").fit_transform([items["time_series"]])[0],
            #     "sliced" : connectome.ConnectivityMeasure(kind="correlation").fit_transform(items["sliced_time_series"])
            # }
            FCs[sub][op][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform([items["time_series"]])[0]

In [9]:
with open("FCs.pkl", "wb") as f:
    pickle.dump(FCs, f)

In [ ]:
# 绘制所有功能连接矩阵
mpl.use("agg")
for sub, ses in FCs.items():
    FCs[sub] = {}
    for op, runs in ses.items():
        os.makedirs(f"fc/{sub}/{op}", exist_ok=True)
        FCs[sub][op] = {}
        for run, fc in runs.items():
            np.fill_diagonal(fc, 0)
            fig_fc, ax_fc = plt.subplots()
            fc_display = plotting.plot_matrix(fc, labels=atlas_labels, figure=fig_fc, colorbar=True, vmax=0.8, vmin=-0.8)
            fig_fc.patch.set_color("white")
            fig_fc.savefig(f"fc/{sub}/{op}/{run}.png", format="png")
            plt.cla()
            plt.clf()
            plt.close("all")

In [ ]:
# 组平均
align_length_preop = 130
align_length_postop = 200
averageFC = {}
for sub, ses in data.items():
    time_series_preop = average_time_series(ses["ses-preop"], align_length_preop)
    time_series_postop = average_time_series(ses["ses-postop"], align_length_postop)
    if time_series_preop is None or time_series_postop is None:
        print(sub, "preop" if time_series_preop is None else "postop")
        continue
    fc = connectome.ConnectivityMeasure(kind="correlation").fit_transform([time_series_preop, time_series_postop])
    averageFC[sub] = {}
    averageFC[sub]["ses-preop"] = fc[0]
    averageFC[sub]["ses-postop"] = fc[1]
    save_fc(fc[0], f"fc/{sub}/average_preop.png")
    save_fc(fc[1], f"fc/{sub}/average_postop.png")

with open("FCs_average.pkl", "wb") as f:
    pickle.dump(averageFC, f)

sub-339 preop
sub-357 preop
sub-369 preop
sub-394 preop
sub-395 preop
sub-403 preop


In [ ]:
# 全体平均
align_length_preop = 130
align_length_postop = 200
total_average_FC = {}
time_series_preop = np.zeros((align_length_preop, 116))
time_series_postop = np.zeros((align_length_postop, 116))
for sub, ses in data.items():
    tmp = average_time_series(ses["ses-preop"], align_length_preop)
    if tmp is None:
        print(sub, "preop")
        continue
    time_series_preop += tmp
    tmp = average_time_series(ses["ses-postop"], align_length_postop)
    if tmp is None:
        print(sub, "postop")
        continue
    time_series_postop += tmp
average_time_series_preop = time_series_preop/len(data)
average_time_series_postop = time_series_postop/len(data)
fc = connectome.ConnectivityMeasure(kind="correlation").fit_transform([average_time_series_preop, average_time_series_postop])
total_average_FC["ses-preop"] = fc[0]
total_average_FC["ses-postop"] = fc[1]
save_fc(fc[0], f"fc/average_preop.png")
save_fc(fc[1], f"fc/average_postop.png")

with open("FCs_toatl_average.pkl", "wb") as f:
    pickle.dump(total_average_FC, f)

sub-339 preop
sub-357 preop
sub-369 preop
sub-394 preop
sub-395 preop
sub-403 preop
